In [22]:
val userHome = System.getProperty("user.home")

println(userHome)

if(userHome.contains("brandl")){
    println("clearing ivy cache")
    java.io.File("$userHome/.ivy2/cache/org.kalasim/").deleteRecursively()
}


C:\Users\brandl
clearing ivy cache


In [1]:
@file:Repository("*mavenLocal")
@file:DependsOn("com.github.holgerbrandl:kalasim:0.7.99")

In [2]:
import org.kalasim.*

In [3]:
//:classpath

In [4]:
%use lets-plot

In [27]:
import org.jetbrains.letsPlot.ggsize
import org.jetbrains.letsPlot.intern.Plot
import org.jetbrains.letsPlot.lets_plot
import krangl.irisData
import krangl.toMap
import org.kalasim.examples.MM1Queue
import org.kalasim.plot.letsplot.display
import org.kalasim.plot.letsplot.displayStateTimeline
import org.kalasim.analysis.InteractionEvent
import java.time.Instant
import java.util.concurrent.TimeUnit


In [28]:
val mm1 = MM1Queue().apply {
    run(100)
}

rho is 0.75. With rho>1 the system would be unstable, because there are more arrivals then the server can serve.


In [8]:
 mm1.traces.filterIsInstance<InteractionEvent>().asDataFrame()

action,curComponent,details,source,logLevel,time
Created,main,capacity=1,server,INFO,.00
create,main,,ComponentGenerator.1,INFO,.00
activate,main,scheduled for .00,ComponentGenerator.1,INFO,.00
run +100.00,main,scheduled for 100.00,main,INFO,.00
hold +.68,ComponentGenerator.1,scheduled for .68,ComponentGenerator.1,INFO,.00
create,ComponentGenerator.1,,Customer.1,INFO,.68
activate,ComponentGenerator.1,scheduled for .68,Customer.1,INFO,.68
hold +.55,ComponentGenerator.1,scheduled for 1.23,ComponentGenerator.1,INFO,.68
Requesting 1.0 from server with priority null a...,Customer.1,null,Customer.1,INFO,.68
null,Customer.1,null,Customer.1,INFO,.68


In [9]:
mm1.server.claimedTimeline.display()

In [10]:
mm1.server.requesters.lengthOfStayMonitor.display()

## MM1 with tick-transformation

In [11]:
val mm1 = MM1Queue()

// redo but with set tick-transform
mm1.tickTransform = OffsetTransform(
    offset = Instant.parse("2021-01-01T00:00:00.00Z"),
    tickUnit = TimeUnit.MINUTES
)

mm1.run(50)

rho is 0.75. With rho>1 the system would be unstable, because there are more arrivals then the server can serve.


{
  "now": "50.00",
  "num_components": 42,
  "queue": [
    "Customer.30",
    "Customer.31",
    "ComponentGenerator.1",
    "Customer.36",
    "Customer.32",
    "Customer.34",
    "Customer.38",
    "Customer.37",
    "Customer.40",
    "Customer.33",
    "Customer.35",
    "Customer.39"
  ]
}

In [30]:
mm1.server.activities.display("MM1 Server Utilization")

In [31]:
mm1.componentGenerator.arrivals.displayStateTimeline("MM1 Server Utilization")

In [32]:
mm1.server.claimedTimeline.display("Claimed Server Capacity")

In [33]:
val customerTimeline =
    mm1.componentGenerator.arrivals.first().statusTimeline

In [34]:

customerTimeline.display("Arrival State Timeline", forceTickAxis = true)


In [36]:
customerTimeline.display("Arrival State Timeline")

## Car

In [32]:
import org.kalasim.*
import org.kalasim.misc.*
import org.kalasim.plot.letsplot.display
import org.kalasim.plot.letsplot.displayStateProportions

In [40]:
class Driver : Resource()

class TrafficLight : State<String>("red")

class Car : Component() {

    val trafficLight = get<TrafficLight>()
    val driver = get<Driver>()

    override fun process() = sequence {
                request(driver) {
                    hold(10)
                }

        request(driver) {
            hold(1.0, description = "driving")

            wait(trafficLight, "green")
        }
    }
}

In [56]:
val sim  = createSimulation(enableConsoleLogger = false) {
    dependency { TrafficLight() }
    dependency { Driver() }
    dependency {
        Car()
    }
    println("simulation setup complete")
}

simulation setup complete


In [57]:
sim.run(20)

In [58]:
sim.get<Driver>().claimedTimeline.display()

In [59]:
sim.get<Driver>().activities.display()

In [60]:
sim.get<TrafficLight>()

TrafficLight.1[red]

In [55]:
sim.get<Car>()

Car.1

In [62]:
sim.get<Car>().statusTimeline.display()